In [78]:
import pandas as pd
import numpy as np
import joblib
import os

from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns

print("Bibliotecas carregadas!")


Bibliotecas carregadas!


In [79]:
df = pd.read_csv("../data/students_clean.csv")

df = df.dropna()
df.head()

,video_id,duration_minutes,title_length,description_length,tags_count,has_thumbnail_custom,video_quality,category,language,has_subtitles,...,comments_count,likes_count,shares_count,playlist_adds,promoted,upload_time,upload_day,total_views,duration_minutes_log,tags_count_log
2,VID01091,47,-0.744253,-1.346645,20.0,0,0,Gaming,Espanhol,Não,...,6943.0,34028,542.0,540,Não,Manhã,Seg-Qui,1420456,0.815401,0.077487
3,VID00388,8,-1.624473,-1.010156,4.0,1,0,Tecnologia,Português,Sim,...,9427.0,42695,3096.0,188,Não,Madrugada,Sábado,329746,-1.471798,-1.554673
4,VID02222,56,1.513703,0.408954,48.0,0,4,Vlogs,Espanhol,Não,...,2993.0,40761,311.0,957,Não,Manhã,Domingo,984985,1.050205,1.041142
5,VID00169,16,1.666785,-1.617300,1.0,1,4,Educação,Português,Não,...,5108.0,33562,4814.0,821,Não,Manhã,Domingo,266178,-0.602829,-2.596795
6,VID01379,34,1.590244,-0.388381,26.0,1,1,Vlogs,Português,Sim,...,5518.0,33442,1515.0,960,Não,Madrugada,Sábado,555307,0.383843,0.363314


4.3 – Separar features e target

In [80]:
X = df.drop(columns=["total_views"])
y = df["total_views"]

4.4 – Carregar divisão dos dados da Etapa 3

In [81]:
X_train = pd.read_csv("../data/X_train.csv")
X_val   = pd.read_csv("../data/X_val.csv")
X_test  = pd.read_csv("../data/X_test.csv")

y_train = pd.read_csv("../data/y_train.csv").squeeze()
y_val   = pd.read_csv("../data/y_val.csv").squeeze()
y_test  = pd.read_csv("../data/y_test.csv").squeeze()

4.5 – Carregar o modelo baseline da Etapa 3

In [82]:
modelo_baseline = joblib.load("modelo_views.pkl")
modelo_baseline

{'preprocess': ColumnTransformer(transformers=[('cat', OneHotEncoder(handle_unknown='ignore'),
                                  Index(['category', 'language', 'has_subtitles', 'promoted', 'upload_time',
        'upload_day'],
       dtype='object')),
                                 ('num', 'passthrough',
                                  Index(['duration_minutes', 'title_length', 'description_length', 'tags_count',
        'has_thumbnail_custom', 'video_quality', 'channel_subscribers',
        'channel_age_months', 'previous_videos_count',
        'avg_upload_frequency_days', 'comments_count', 'likes_count',
        'shares_count', 'playlist_adds', 'duration_minutes_log',
        'tags_count_log'],
       dtype='object'))]),
 'model': LinearRegression()}

4.6 – Avaliação do baseline (antes da otimização)


In [83]:
import joblib
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Carregar o bundle da Etapa 3
bundle = joblib.load("modelo_views.pkl")

preprocess = bundle["preprocess"]
modelo_baseline = bundle["model"]

# Aplicar preprocessamento ao conjunto de validação
X_val_transf = preprocess.transform(X_val)

# Predições
pred_val_baseline = modelo_baseline.predict(X_val_transf)

# Métricas
mae_base = mean_absolute_error(y_val, pred_val_baseline)

# RMSE compatível com todas as versões:
rmse_base = mean_squared_error(y_val, pred_val_baseline) ** 0.5

mae_base, rmse_base

(247329.196597814, 478596.25958482904)

4.7 – Definir grid para Random Search

In [84]:
param_grid = {
    "n_estimators": [200, 400, 800, 1200],
    "max_depth": [5, 10, 20, None],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4],
    "max_features": ["sqrt", "log2"]
}

modelo = RandomForestRegressor(random_state=42)

4.8 – Executar Random Search

In [85]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
import numpy as np

# Criar pipeline completo (preprocess + modelo)
pipeline = Pipeline([
    ("preprocess", preprocess),
    ("model", modelo)
])

# Definir grid compatível com RandomForest
param_grid = {
    "model__n_estimators": [100, 200, 300, 400],
    "model__max_depth": [5, 10, 15, 20, None],
    "model__min_samples_split": [2, 5, 10],
    "model__min_samples_leaf": [1, 2, 4],
    "model__bootstrap": [True, False],
}

# Randomized Search
random_search = RandomizedSearchCV(
    estimator=pipeline,
    param_distributions=param_grid,
    n_iter=20,
    cv=3,
    scoring="neg_mean_absolute_error",
    random_state=42,
    n_jobs=-1
)
random_search.fit(X_train, y_train)

,estimator,Pipeline(step...m_state=42))])
,param_distributions,"{'model__bootstrap': [True, False], 'model__max_depth': [5, 10, ...], 'model__min_samples_leaf': [1, 2, ...], 'model__min_samples_split': [2, 5, ...], ...}"
,n_iter,20
,scoring,'neg_mean_absolute_error'
,n_jobs,-1
,refit,True
,cv,3
,verbose,0
,pre_dispatch,'2*n_jobs'
,random_state,42
,error_score,nan


Heatmap da matriz de correlação entre todas as variáveis numéricas do conjunto de dados. As cores representam a intensidade e a direção das correlações, onde tons próximos ao vermelho indicam correlação positiva forte e tons próximos ao azul indicam correlação negativa forte. A visualização permite identificar padrões de dependência entre atributos, possíveis redundâncias, multicolinearidade e relações que podem influenciar o desempenho dos modelos de Machine Learning.”

4.9 – Avaliação do modelo otimizado

In [86]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

pred_val_opt = best_model.predict(X_val)

mae_opt = mean_absolute_error(y_val, pred_val_opt)

# RMSE compatível com qualquer versão do sklearn
mse_opt = mean_squared_error(y_val, pred_val_opt)
rmse_opt = np.sqrt(mse_opt)

r2_opt = r2_score(y_val, pred_val_opt)

print("📌 Desempenho do Modelo Otimizado (Validação):")
print(f"MAE:  {mae_opt:.2f}")
print(f"RMSE: {rmse_opt:.2f}")
print(f"R²:   {r2_opt:.4f}")

📌 Desempenho do Modelo Otimizado (Validação):
MAE:  117910.22
RMSE: 258831.44
R²:   0.8310


In [92]:
import joblib

# Carregar o pacote salvo na Etapa 3
bundle = joblib.load("modelo_views.pkl")

# Extrair preprocessador e modelo baseline
preprocessador = bundle["preprocess"]
modelo_baseline = bundle["model"]

print("Bundle carregado com sucesso!")
print("Objetos disponíveis:", bundle.keys())

Bundle carregado com sucesso!
Objetos disponíveis: dict_keys(['preprocess', 'model'])


4.10 – Treinar modelo final (treino + validação)

In [93]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor

bundle = joblib.load("modelo_views.pkl")

preprocessador = bundle["preprocess"]

X_train_final = pd.concat([X_train, X_val], axis=0)
y_train_final = pd.concat([y_train, y_val], axis=0)

best_params = random_search.best_params_
print("Hiperparâmetros escolhidos:", best_params)

modelo_final = Pipeline([
    ("preprocess", preprocessador),
    ("model", RandomForestRegressor(
        n_estimators=best_params["model__n_estimators"],
        max_depth=best_params["model__max_depth"],
        min_samples_split=best_params["model__min_samples_split"],
        random_state=42
    ))
])

modelo_final.fit(X_train_final, y_train_final)

print("Modelo final treinado com sucesso!")


Hiperparâmetros escolhidos: {'model__n_estimators': 300, 'model__min_samples_split': 5, 'model__min_samples_leaf': 2, 'model__max_depth': 20, 'model__bootstrap': True}
Modelo final treinado com sucesso!


4.11 – Avaliação final no conjunto de teste

In [94]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

pred_test = modelo_final.predict(X_test)

mae_test = mean_absolute_error(y_test, pred_test)
rmse_test = np.sqrt(mean_squared_error(y_test, pred_test))  # sem parâmetro squared
r2_test = r2_score(y_test, pred_test)

print("📌 Desempenho Final no TESTE (Modelo Otimizado):")
print(f"MAE  = {mae_test:.4f}")
print(f"RMSE = {rmse_test:.4f}")
print(f"R²   = {r2_test:.4f}")

📌 Desempenho Final no TESTE (Modelo Otimizado):
MAE  = 117626.6413
RMSE = 321955.0861
R²   = 0.7584


4.12 – Salvar modelo final

In [95]:
import os
import joblib

os.makedirs("../models", exist_ok=True)

joblib.dump(modelo_final, "../models/modelo_final.joblib")

print("Modelo salvo com sucesso em: ../models/modelo_final.joblib")


Modelo salvo com sucesso em: ../models/modelo_final.joblib
